---
# Automatically generate new Python files from the Spoonacular API info page
---

In [1]:
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Start the Selenium browser (Chrome)

In [2]:
# Start a new instance of Chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
browser = webdriver.Chrome('/usr/local/bin/chromedriver', chrome_options=chrome_options)
url = "https://spoonacular.com/food-api"
browser.get(url)
time.sleep(1)

# Scroll down to list of endpoints (necessary?)
elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 8
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    no_of_pagedowns-=1
    
print("Waiting for endpoints to load...")
time.sleep(5)

Waiting for endpoints to load...


## Scrape documentation for each endpoint

In [167]:
def getEndpointInfo(save=True, filename='spoonacular_api_info'):
    endpoints = []
    m,p = 'mashape-', 'parameter-'
    regex = re.compile(r'required|optional', re.VERBOSE)    
    doc = browser.find_element_by_class_name(m+'doc')
    groups = doc.find_elements_by_class_name(m+'group')
    num_eps = len(doc.find_elements_by_class_name(m+'endpoint'))

    n = 0
    for group in groups:
        print(10*'-' + group.find_element_by_class_name(m+'group-header').text + 10*'-')
        for ep in group.find_elements_by_class_name(m+'endpoint'):            
            if n!= 36: # Kludge for bug on Spoonacular site
                ep.click()  # Must click on endpoint to expand its documentation
            time.sleep(0.5)
            print('**'+ep.find_element_by_class_name(m+'endpoint-name').text+'**')
                                                
            # Get info on the endpoint name and route
            endpoint = {}
            endpoint['name'] = ep.find_element_by_class_name(m+'endpoint-name').text
            print('-'*30 + '\n({n}/{t}) "{name}"'.format(name=endpoint['name'], n=n+1, t=num_eps))
            content = ep.find_element_by_class_name(m+'endpoint-content')
            route = content.find_element_by_class_name(m+'endpoint-route').text.split(' /')
            endpoint['method'] = route[0]
            endpoint['uri'] = route[1]
            endpoint['category'] = group.find_element_by_class_name(m+'group-header').text
            endpoint['description'] = ep.find_element_by_tag_name('p').text
            time.sleep(1.5)            
            
            # Get parameter info for the endpoint
            parameters = {'route_params': [], 'params': []}
            assign_to_route_params = True        
            for div in content.find_elements_by_tag_name('div'):
                if div.get_attribute('class') == 'mashape-parameter-header':
                    print(div.text)
                    if div.text == 'Parameters':
                        assign_to_route_params = False

                if div.get_attribute('class') == 'mashape-parameter':                
                    parameter = {}    
                    if div.text != '':
                        for s in div.find_elements_by_tag_name('span'):
                            var = s.get_attribute('class').split(m+p,1)[-1].lower()
                            if 'condition' in var:
                                var = 'condition'
                                val = regex.search(s.text).group()
                            elif 'example' in var:
                                val = s.text.split('Example:',1)[-1]
                            else:
                                val = s.text
                            parameter[var] = val
                        print("\t{name}: {type}".format(name=parameter['name'], type=parameter['type']))
                    else:
                        print("\t\t***MISSING PARAMETER***")
                        parameter = {'MISSING': True}                

                    if assign_to_route_params:
                        parameters['route_params'].append(parameter)
                    else:
                        parameters['params'].append(parameter)
                        
                endpoint['parameters'] = parameters

                # Get info on example endpoint response
                try:
                    endpoint['response'] = content.find_element_by_class_name(m+'example').text
                except:
                    endpoint['response'] = None                                    

            # Add info on current endpoint to list of all endpoints            
            endpoints.append(endpoint)
            n += 1
            
    if save:
        with open(filename + '.json', 'w') as outfile:
            json.dump(endpoints, outfile)                
                
    return endpoints                               
                                    

In [151]:
info[-1]['parameters']

{'route_params': [{'name': 'includeNutrition',
   'type': 'BOOLEAN',
   'condition': 'optional',
   'description': 'Whether nutrition data should be added to correctly parsed ingredients.',
   'example': 'false'}],
 'params': [{'name': 'ingredientList',
   'type': 'STRING',
   'condition': 'required',
   'description': 'The ingredient list of the recipe, one ingredient per line.',
   'example': '3 oz pork shoulder'},
  {'name': 'servings',
   'type': 'NUMBER',
   'condition': 'required',
   'description': 'The number of servings that you can make from the ingredients.',
   'example': '2'}]}

In [156]:
info = getEndpointInfo()

----------Extract----------
**Analyze a Recipe Search Query**
------------------------------
(1/53) "Analyze a Recipe Search Query"
Route Parameters
	q: STRING
**Analyze Recipe Instructions**
------------------------------
(2/53) "Analyze Recipe Instructions"
Parameters
	instructions: STRING
**Detect Food in Text**
------------------------------
(3/53) "Detect Food in Text"
Parameters
	text: STRING
**Extract Recipe from Website**
------------------------------
(4/53) "Extract Recipe from Website"
Route Parameters
	forceExtraction: BOOLEAN
	url: STRING
**Parse Ingredients**
------------------------------
(5/53) "Parse Ingredients"
Route Parameters
	includeNutrition: BOOLEAN
Parameters
	ingredientList: STRING
	servings: NUMBER


In [44]:
def getEndpointInfo(save=True):
    endpoints = []
    m,p = 'mashape-', 'parameter-'
    regex = re.compile(r'required|optional', re.VERBOSE)    
    doc = browser.find_element_by_class_name(m+'doc')
    groups = doc.find_elements_by_class_name(m+'group')
    
    group_names, list_of_endpoints = [],[]
    for group in groups:
        group_endpoints = group.find_elements_by_class_name(m+'endpoint')
        group_names += len(group_endpoints)*[group.find_element_by_class_name(m+'group-header').text]
        list_of_endpoints += group_endpoints
                        
    del(group_names[36])
    del(list_of_endpoints[36]) # Bad element on Spoonacular site
    num_eps = len(list_of_endpoints)
    print("Found {num} endpoints to scrape.".format(num=num_eps))
    
    n = 0
    for ep, group_name in zip(list_of_endpoints[:5], group_names[:5]):
        ep.click()
        time.sleep(0.5)
        endpoint = {}
        
        # Get info on the endpoint name and route
        endpoint['name'] = ep.find_element_by_class_name(m+'endpoint-name').text
        print('-'*30 + '\n({n}/{t}) "{name}"'.format(name=endpoint['name'], n=n+1, t=num_eps))
        content = ep.find_element_by_class_name(m+'endpoint-content')
        route = content.find_element_by_class_name(m+'endpoint-route').text.split(' /')
        endpoint['method'] = route[0]
        endpoint['uri'] = route[1]
        endpoint['category'] = group_name
        endpoint['description'] = ep.find_element_by_tag_name('p').text
        time.sleep(2)
        
        # Get info on the endpoint parameters
        parameters = []
        print("Params:")
        params = content.find_elements_by_class_name(m+'parameter')
        for param in params:
            parameter = {}    
            if param.text != '':
                for s in param.find_elements_by_tag_name('span'):
                    var = s.get_attribute('class').split(m+p,1)[-1].lower()
                    if 'condition' in var:
                        var = 'condition'
                        val = regex.search(s.text).group()
                    elif 'example' in var:
                        val = s.text.split('Example:',1)[-1]
                    else:
                        val = s.text
                    parameter[var] = val
                print("\t{name}: {type}".format(name=parameter['name'], type=parameter['type']))
            else:
                print("\t\t***MISSING PARAMETER***")
                parameter = {'MISSING': True}
            parameters.append(parameter)            
        endpoint['parameters'] = parameters
        
        # Get info on example endpoint response
        try:
            endpoint['response'] = content.find_element_by_class_name(m+'example').text
        except:
            endpoint['response'] = None
            
        # Add info on current endpoint to list of all endpoints            
        endpoints.append(endpoint)
                
        if save and ((n)%10==0 or n+1==len(list_of_endpoints)):
            filename = 'spoonacular_api_names_and_endpoints'
            with open(filename + '.json', 'w') as outfile:
                json.dump(endpoints, outfile)                
                
        n+=1

    return endpoints                   

In [60]:
all_info = json.load(open('spoonacular_api_names_and_endpoints.json', 'rb'))

In [45]:
all_info = getEndpointInfo()
all_info[-1]

Found 52 endpoints to scrape.
------------------------------
(1/52) "Analyze a Recipe Search Query"
Params:
	q: STRING
------------------------------
(2/52) "Analyze Recipe Instructions"
Params:
	instructions: STRING
------------------------------
(3/52) "Detect Food in Text"
Params:
	text: STRING
------------------------------
(4/52) "Extract Recipe from Website"
Params:
	forceExtraction: BOOLEAN
	url: STRING
------------------------------
(5/52) "Parse Ingredients"
Params:
	includeNutrition: BOOLEAN
	ingredientList: STRING
	servings: NUMBER


{'name': 'Parse Ingredients',
 'method': 'POST',
 'uri': 'recipes/parseIngredients',
 'category': 'Extract',
 'description': 'Extract an ingredient from plain text.',
 'parameters': [{'name': 'includeNutrition',
   'type': 'BOOLEAN',
   'condition': 'optional',
   'description': 'Whether nutrition data should be added to correctly parsed ingredients.',
   'example': 'false'},
  {'name': 'ingredientList',
   'type': 'STRING',
   'condition': 'required',
   'description': 'The ingredient list of the recipe, one ingredient per line.',
   'example': '3 oz pork shoulder'},
  {'name': 'servings',
   'type': 'NUMBER',
   'condition': 'required',
   'description': 'The number of servings that you can make from the ingredients.',
   'example': '2'}],
 'response': '[{"id":10072,"original":"3 oz pork shoulder","name":"pork shoulder","amount":3.0,"unitShort":"oz","unitLong":"ounces","aisle":"Meat","image":"https://spoonacular.com/cdn/ingredients_100x100/pork-shoulder.jpg","meta":[]}]'}

# Generate the Python wrapper from scraped API documentation

In [63]:
all_info[0]

{'name': 'Analyze a Recipe Search Query',
 'method': 'GET',
 'uri': 'recipes/queries/analyze',
 'parameters': [{'name': 'q',
   'type': 'STRING',
   'condition': 'required',
   'description': 'The recipe search query.',
   'example': 'salmon with fusilli and no nuts'}],
 'response': '{"ingredients":[{"name":"fusilli","include":true,"image":"fusilli.jpg"},{"name":"nuts mixed","include":false,"image":"nuts-mixed.jpg"}],"dishes":[{"name":"salmon","image":"https://spoonacular.com/cdn/ingredients_100x100/salmon.jpg"}],"modifiers":[],"cuisines":[]}'}

In [70]:
all_info[6]['parameters'][0]

{'name': 'number',
 'type': 'NUMBER',
 'condition': 'optional',
 'description': 'The number of results between [1,25].',
 'example': '10'}

In [91]:
def formatMethodName(name):
    return name.lower().replace(' ','_')    

In [72]:
name = all_info[0]['name']
name

'Analyze a Recipe Search Query'

In [103]:
def parse_ingredients(self, ingredientList, servings=1, includeNutrition=False):
    """ Extract an ingredient from plain text
        https://market.mashape.com/spoonacular/recipe-food-nutrition#parse-ingredients
    """
    endpoint = "recipes/parseIngredients"
    query = {"ingredientList": ingredientList, "servings": servings}
    params = {"includeNutrition": includeNutrition}
    return self._make_request(endpoint, method='POST', query_=query, params_=params)


In [215]:
def formatArgList(varNames):
    if len(varNames) == 0:
        return "self"
    return "self, " + ", ".join(varNames)

def writeEndpointMethod(name, docstr, argList, method, uri, indent=0):
    txt = 'def {name}({args}):\n'.format(name=formatMethodName(name), args=formatArgList(argList))
    txt += '\t""" {doc}\n'.format(doc=docstr)
    txt += '\t\thttps://market.mashape.com/spoonacular/recipe-food-nutrition#{name}\n'.format(name=name.lower().replace(' ','-'))
    txt += '\t"""\n'
    txt += '\tendpoint = "{uri}"\n'.format(uri=uri)
    txt += '\tquery = {{}}\n'
    txt += '\tparams = {{}}\n'
    txt += '\treturn self._make_request(endpoint, method="{m}", query_=query)'.format(m=method)
    
    # Indent the method (optional) and replace tabs with spaces
    final = ""
    for line in txt.split('\n'):
        final += "\t"*indent + line + '\n'
    final = final.replace('\t', 4*' ')
    return final

In [214]:
n = 3
m = writeEndpointMethod(all_info[n]['name'],
                        "Here is what the method does",
                        [p['name'] for p in all_info[n]['parameters']],
                        all_info[n]['method'],
                        all_info[n]['uri'],
                        indent=0)
print(m)
m

def extract_recipe_from_website(self, forceExtraction, url):
    """ Here is what the method does
        https://market.mashape.com/spoonacular/recipe-food-nutrition#extract-recipe-from-website
    """
    endpoint = "recipes/extract"
    query = {{}}
    params = {{}}
    return self._make_request(endpoint, method="GET", query_=query)



'def extract_recipe_from_website(self, forceExtraction, url):\n    """ Here is what the method does\n        https://market.mashape.com/spoonacular/recipe-food-nutrition#extract-recipe-from-website\n    """\n    endpoint = "recipes/extract"\n    query = {{}}\n    params = {{}}\n    return self._make_request(endpoint, method="GET", query_=query)\n'

In [180]:
all_info[3]

{'name': 'Extract Recipe from Website',
 'method': 'GET',
 'uri': 'recipes/extract',
 'parameters': [{'name': 'forceExtraction',
   'type': 'BOOLEAN',
   'condition': 'optional',
   'description': 'If true, the extraction will be triggered no matter whether we know the recipe already. Use that only if information is missing as this operation is slower.',
   'example': 'false'},
  {'name': 'url',
   'type': 'STRING',
   'condition': 'required',
   'description': 'The URL of the recipe page.',
   'example': 'http://www.melskitchencafe.com/the-best-fudgy-brownies/'}],
 'response': None}

In [71]:
max_width = 79
def formatArgListAndPathFromURI(uri, format_for_tests=False):
    regex = re.compile('(?!:format)(:[\w_]+)', re.VERBOSE) 
    
    # Identify the arguments to the URI, excluding :format
    parameters = [s.strip(':') for s in regex.findall(uri) if s != '']
    if format_for_tests:
        arg_list = ''
        for p in parameters:
            arg_list += "self.{p}, ".format(p=p)
        if parameters:
            arg_list = arg_list.strip(', ')
    else:
        arg_list = ", ".join(parameters)
    
    # Create the path string, allowing user to substitute in arguments
    path = (regex.sub('{\g<1>}', uri).split(':format')[0] + '"').replace('{:','{')
    print(path)
    
    # Make sure dates are formatted properly
    if 'year' in parameters:
        path = path.replace('{year}', '{year:4d}')
    if 'month' in parameters:
        path = path.replace('{month}', '{month:02d}')
    if 'day' in parameters:
        path = path.replace('{day}', '{day:02d}')        
    
    # Append .format() to the end of the path string
    format_suffix = ""
    for p in parameters:
        format_suffix += "{arg}={val}, ".format(arg=p, val=p)
    format_suffix = ".format({})".format(format_suffix.strip(', '))    
    path += format_suffix
#     path = path.split('/',3)[-1]
    
    # Comply to max width of lines
    if len(path) > max_width-8:
        regex = re.compile(r'\.format\(', re.VERBOSE)
        path = regex.sub(r'.format(\n\t\t\t', path)        
    path = '"' + path
    
    return arg_list, path

def formatPathFromURI(uri):
    parameters = [s.strip(':') for s in re.findall(r'(:[\w_]+)*', uri) if s != ''][:-1]

def formatDocString(doc, mw=79):
    if len(doc) > mw-7:
        idx_cut = (mw-len(doc[mw::-1].split(' ')[0])) # kludge        
        doc = doc[:idx_cut] + '\n\t\t\t' + formatDocString(doc[idx_cut:].strip(' '), mw)
    return doc

def paramValsAreComplete(endpoint):
    for val in endpoint['defaults'].values():
        if val == '':
            return False
    return True

def formatDefaultParamVals(endpoints):
    assert type(endpoints)==list, 'Must provide a list of endpoints'
    varlist = ''
    used_vars = []
    
    for ep in endpoints:
        params = ep['parameters']
        for param in params:
            var,val = param['name'], param['example']            
            if var not in used_vars and val != '':
                if var in ['year', 'month', 'day']:                    
                    varlist += '\t\tcls.{var} = {val}\n'.format(var=var, val=int(val))
                else:
                    varlist += '\t\tcls.{var} = "{val}"\n'.format(var=var, val=val)
                used_vars.append(var)
    return varlist

def formatClassName(name):
    name = name.replace(' Trial','').replace('Official','')
    name = name.replace(' ','').replace('.','_').replace('(','')
    name = name.replace(')','').replace('-','_').replace(',','_')
    return name
    
def writeToPythonFile(api_info):
    """Write methods based on the endpoint attributes
    :param api_info: (dict) Contains API name, description and list of endpoints
    """

    # --------------------------------------------------------
    #    Write the methods
    # --------------------------------------------------------
                
    # Class name    
    class_name = formatClassName(api_info['name'])
    regex = re.compile(r'v[\d_]+', re.IGNORECASE)
    class_name = regex.sub('', class_name)
        
    # Add comment heading and import statement
    txt = "# Sportradar APIs\n# Copyright 2018 John W. Miller\n# See LICENSE for details.\n\n"
    txt += "from sportradar.api import API\n\n\n"
           
    # Add class name to file   
    txt += "class {}(API):\n\n".format(class_name)
    
    # Add __init__ function to class
    txt += "\tdef __init__(self, api_key, format_='json', timeout=5, sleep_time=1.5):\n"
    txt += "\t\tsuper().__init__(api_key, format_, timeout, sleep_time)\n\n"
    txt = txt.replace('\t', 4*' ')
            
    # Write the method, including arguments, doc string, and URI path    
    endpoints = api_info['endpoints']    
    for n,ep in enumerate(endpoints):
        mn = 'get_' + formatClassName(ep['name']).lower()
        doc = formatDocString(ep['description'], max_width)
        doc = doc + '"""' if len(doc) < max_width-8 else doc + '\n\t\t"""'
        args, path = formatArgListAndPathFromURI(ep['uri'])                
        arglist = 'self, {a}'.format(a=args) if args != '' else 'self'                
        
        # Assemble the method string
        txt += '\tdef {method_name}({args}):\n'.format(method_name=mn, args=arglist)
        txt += '\t\t"""{doc}\n\t\tpath = {path}\n'.format(doc=doc, path=path)
        txt += '\t\treturn self._make_request(path)\n\n'
        txt = txt.replace('\t', 4*' ')
                        
    print('-'*20)
    print(api_info['name'])
#     print(path)
#     print(txt)
    
    # Save to Python file
    filename = class_name + '.py'
    with open(filename, 'w+') as pyfile:
        pyfile.write(txt)
        
        
    # --------------------------------------------------------
    #    Write the tests
    # --------------------------------------------------------
    
    # Write the test front matter
    txt = "import os\nimport unittest\nfrom sportradar import {}\n\n".format(class_name)
    txt += '# Import API keys from environment variables\n'
    txt += 'api_key_name = "SPORTRADAR_API_KEY_{}"\n'.format(class_name.upper())
    txt += 'api_key = os.environ.get(api_key_name, None)\n'
    txt += 'assert api_key is not None, "Must declare environment variable: {key_name}".format(\n'
    txt += '\tkey_name=api_key_name)\n'
    txt += 'api = {}.{}(api_key, format_="json", timeout=5, sleep_time=1.5)\n\n\n'.format(
        class_name, class_name)
    txt += 'class TestAPI(unittest.TestCase):\n\n'
    txt += '    @classmethod\n    def setUpClass(cls):\n'
    txt += '\t\tprint("\\n---------------------\\nSetting up {} tests...\\n".format("{}"))\n'.format("{}", class_name)
    txt += '\t\tcls.auth = api_key\n\t\tcls.api = api\n'
    txt += '{defaults}\n'.format(defaults=formatDefaultParamVals(endpoints))
       
    # Write the test methods
    # Write the method, including arguments, doc string, and URI path    
    for n,ep in enumerate(endpoints):        
        if paramValsAreComplete(ep):        
            mn = 'test_get_' + formatClassName(ep['name']).lower()
            doc = '"""Test the {} GET query"""'.format(ep['name'].lower())    
            args, path = formatArgListAndPathFromURI(ep['uri'], format_for_tests=True)
            arglist = '{a}'.format(a=args) if args != '' else ''
            print(arglist)

            # Assemble the method string    
            txt += '\tdef {method_name}(self):\n\t\t{doc}\n\t\tmsg = "Response status is not 200"\n'.format(
                    method_name=mn, doc=doc)
            txt += '\t\tresponse = self.api.{}({})\n'.format(
                mn.split('test_')[1], arglist.split('self, ', 1)[-1])
            
            if n == len(endpoints)-1:            
                txt += '\t\tself.assertEqual(response.status_code, 200, msg)\n'
            else:
                txt += '\t\tself.assertEqual(response.status_code, 200, msg)\n\n'
            txt = txt.replace('\t', 4*' ')
        
    # Save to Python file
    filename = 'test_' + class_name + '.py'
    with open(filename, 'w+') as pyfile:
        pyfile.write(txt)        


In [ ]:
writeToPythonFile()